<a href="https://colab.research.google.com/github/Joab-S/SimulacaoEstocasticaUFC/blob/main/SistemaDeSeguros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install simpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import simpy as sp
import numpy as np

In [12]:
# Variáveis de Sistema:
N, C = 10, 1200
client_number = N
capital = C
last_event_time = 0

# Parâmetros do Sistema:
arrival_fee = 1/4                   # Em média, chegam 4 pessoas por mês
exit_fee = 12/client_number         # Em média, um cliente permanece no seguro por 12 meses
occurrence_fee = 3/client_number    # Em média, um cliente aciona o seguro a cada 3 meses
monthly_payment = 40
occurrence_value = 100
occurrence_variance = 40
limit = 120                         # Limite de simulação para 120 meses

In [ ]:
def partialMonthlyPayment(env, n):
  global capital, last_event_time

  capital += n * (env.now() - last_event_time) * monthly_payment
  print("Mensalidade paga, Capital =", capital)

In [15]:
def arrivalSystem(env):
  global client_number, last_event_time
  
  while True:
    client_number += 1,
    print("Um novo cliente chegou. n: ", client_number)
    
    yield env.timeout(np.random.exponential(arrival_fee))
    
    partialMonthlyPayment(env, client_number)

    last_event_time = env.now()

In [ ]:
def exitSystem(env):
  ...

In [ ]:
def occurrence(env):
  ...

In [ ]:
def initSystem(env):
  env.process(arrivalSystem(env))
  if (client_number > 0):
    env.process(occurrence(env))
    env.process(exitSystem(env))

In [ ]:
def minCapitalSystem():
  global client_number
  env = sp.Environment()
  initSystem(env)
  env.run(until=limit)
  print("Fim da execução!")

  min = 0
  return min